# Домашнее задание 
# Математика — Домашнее задание 8
# А.&#8201;Д. Егоров<h1/>

In [1]:
import numpy as np
import scipy as sp 

import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns

from collections import Counter

import datetime
from dateutil.parser import parse

plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
%matplotlib inline

Данные взяты отсюда: https://www.kaggle.com/datasets/2b61e7d5e0a1646ea26617c1f5cd452d39bffe3001ba6a5bc96d919bb31fd221

In [2]:
df_users = pd.read_csv("./vodclickstream_uk_movies_03.csv")
df_users.head(5)

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


In [3]:
df_users.shape

(671736, 8)

In [4]:
len(df_users["user_id"].unique())

161918

Посмотрим какие данные присутствуют в таблице.
Точнее, взклянем на размах по дате в данных.

In [5]:
df_users = df_users.sort_values(by="datetime")


info = f"""
    Число записей: 
    2017: {df_users[df_users["datetime"].str.contains("2017", regex=False)].shape[0]},
    2018: {df_users[df_users["datetime"].str.contains("2018", regex=False)].shape[0]},
    2019: {df_users[df_users["datetime"].str.contains("2019", regex=False)].shape[0]}.

    Даты: 
    Первая запись: {df_users["datetime"].iloc[0]},
    Последняя запись: {df_users["datetime"].iloc[-1]}.
"""

print(info)


    Число записей: 
    2017: 247396,
    2018: 273975,
    2019: 150365.

    Даты: 
    Первая запись: 2017-01-01 00:02:21,
    Последняя запись: 2019-06-30 23:59:20.



Датасет большой, предлагается урезать его в целях экономии времени на выполнении обращений к БД.
Возьмем данные только за 2018 год.

In [6]:
df_users_2017 = df_users[df_users["datetime"].str.match("2017-*")].sort_values(
    by="datetime"
)
df_users_2018 = df_users[df_users["datetime"].str.match("2018-*")].sort_values(
    by="datetime"
)

In [7]:
df_users_2018.head(5)

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
247686,306459,2018-01-01 00:02:26,0.0,Looper,"Action, Drama, Sci-Fi, Thriller",2012-09-28,4718f9963c,dbf944ee37
247653,306426,2018-01-01 00:03:14,1080.0,All Hail King Julien: New Year's Eve Countdown,NOT AVAILABLE,NOT AVAILABLE,3e477ccb02,6e1394fef9
248550,307323,2018-01-01 00:06:38,158.0,Madagascar 3: Europe's Most Wanted,"Animation, Adventure, Comedy, Family",2012-06-08,22d2b55a4b,91a6463280
247570,306343,2018-01-01 00:09:11,0.0,Spotlight,"Biography, Crime, Drama",2015-11-20,f0b97d8d31,da3c14f86a
247438,306211,2018-01-01 00:09:16,0.0,Madagascar: Escape 2 Africa,"Animation, Adventure, Comedy, Family",2008-11-07,cec66d971b,91a6463280


Откинем часы-минуты-секунды

In [8]:
df_users_2018["datetime"] = df_users_2018["datetime"].apply(lambda x: x[:11])

## Задание

Необходимо взять (или сгенерировать самому) датасет, в котором для достаточно большой целевой аудитории (абоненты или лояльные клиенты) представлены еженедельные метрики (тут на ваш выбор). Например, в течение 2-3 месяцев вы собирали, накапливали ваши данные. Далее используйте различные техники для проверки распределений ваших метрик (1 метрика - непрерывная, 2-я дискретная [можно взять бинарную]).

Пример. У меня есть 1 миллион юзеров Киона. Собираю по ним метрики за 12 недель (суммарное время просмотра за каждую неделю и флаг просмотра хотя бы одного фильма из какой-то категории). Далее сравниваем распределения метрик в динамике по разным неделям и делаем выводы о появлении отклонения (распределение начало изменяться или нет).

## Непрерывная метрика

В качестве непрерывной метрики выберем время просмотра.

In [9]:
df_duration_per_day = df_users_2018.groupby(by="datetime", as_index=False).agg(
    {"duration": "sum"}
)

df_duration_per_day.head(5)

,datetime,duration
0,2018-01-01,29393571.0
1,2018-01-02,26624028.0
2,2018-01-03,21895037.0
3,2018-01-04,21055964.0
4,2018-01-05,19962338.0


In [10]:
df_duration_per_day["datetime"] = pd.to_datetime(df_duration_per_day["datetime"])
df_duration_per_day["week"] = df_duration_per_day["datetime"].dt.isocalendar().week
df_duration_per_day = df_duration_per_day.drop(index=364)
df_duration_per_day.head(10)

,datetime,duration,week
0,2018-01-01,29393571.0,1
1,2018-01-02,26624028.0,1
2,2018-01-03,21895037.0,1
3,2018-01-04,21055964.0,1
4,2018-01-05,19962338.0,1
5,2018-01-06,20576099.0,1
6,2018-01-07,22676917.0,1
7,2018-01-08,18479878.0,2
8,2018-01-09,27016983.0,2
9,2018-01-10,18730183.0,2


### Тест Колмогорова-Смирнова
Рассмотрим тест Колмогорова-Смирнова на данных по распределению времени просмотра понедельно (возьмем первые 20 недель):

Сначала будем сравнивать новую неделю с предыдущей, т.е. 1 и 2, 2 и 3, и т.д.

In [11]:
data_ks_2samp_new_prev = []
for j in range(1, 20):
    res = sp.stats.ks_2samp(
        df_duration_per_day[(df_duration_per_day["week"] == j)]["duration"],
        df_duration_per_day[(df_duration_per_day["week"] == j + 1)]["duration"],
    )
    data_ks_2samp_new_prev.append([j, j + 1, res.pvalue])

data_ks_2samp_new_prev = pd.DataFrame(
    data_ks_2samp_new_prev, columns=["week_orig", "week_new", "pvalue"]
)
data_ks_2samp_new_prev["test_passed"] = data_ks_2samp_new_prev["pvalue"].apply(
    lambda x: True if x > 0.05 else False
)
data_ks_2samp_new_prev

/tmp/ipykernel_78756/673165030.py:3: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  res = sp.stats.ks_2samp(


,week_orig,week_new,pvalue,test_passed
0,1,2,0.575175,True
1,2,3,0.962704,True
2,3,4,0.212121,True
3,4,5,0.575175,True
4,5,6,0.962704,True
5,6,7,0.575175,True
6,7,8,0.575175,True
7,8,9,0.053030,True
8,9,10,0.212121,True
9,10,11,0.053030,True


Теперь будем сравнивать с эталонной неделей. Выберем за эталон первую неделю.

In [12]:
data_ks_2samp_ref_prev = []
ref_week = 1
for j in range(1, 20):
    res = sp.stats.ks_2samp(
        df_duration_per_day[(df_duration_per_day["week"] == ref_week)]["duration"],
        df_duration_per_day[(df_duration_per_day["week"] == j + 1)]["duration"],
    )
    data_ks_2samp_ref_prev.append([ref_week, j + 1, res.pvalue])

data_ks_2samp_ref_prev = pd.DataFrame(
    data_ks_2samp_ref_prev, columns=["week_orig", "week_new", "pvalue"]
)
data_ks_2samp_ref_prev["test_passed"] = data_ks_2samp_ref_prev["pvalue"].apply(
    lambda x: True if x > 0.05 else False
)
data_ks_2samp_ref_prev

,week_orig,week_new,pvalue,test_passed
0,1,2,0.575175,True
1,1,3,0.212121,True
2,1,4,0.962704,True
3,1,5,0.575175,True
4,1,6,0.212121,True
5,1,7,0.212121,True
6,1,8,0.575175,True
7,1,9,0.053030,True
8,1,10,0.212121,True
9,1,11,0.962704,True


Видно, что иногда наблюдаются выбросы, но их количество незначительно. В целом делаем вывод, что распределение времени просмотра не зависит от недели. Все плюс-минус одниково.

### Дивергенция Кульбака-Лейбнера:

Проведем те же расчеты, но посчитаем дивергенцию Кульбака-Лейбнера и вычислим индекс стабильности популяции.


Аналогично сравниваем 1 и 2 недели, 2 и 3 и т.д.

Единственное отличие будет в том, что в данном случае на вход функции будем подавать отнормированные данные: нормируем по суммарному времени просмотра за неделю.

In [13]:
kl = []
data_kl = []
for j in range(1, 20):
    p = (
        df_duration_per_day[(df_duration_per_day["week"] == j)]["duration"].values
        / df_duration_per_day[(df_duration_per_day["week"] == j)][
            "duration"
        ].values.sum()
    )
    q = (
        df_duration_per_day[(df_duration_per_day["week"] == j + 1)]["duration"].values
        / df_duration_per_day[(df_duration_per_day["week"] == j + 1)][
            "duration"
        ].values.sum()
    )
    res = sp.special.kl_div(p, q).sum()
    res += sp.special.kl_div(q, p).sum()
    data_kl.append([j, j + 1, res])
df_kl_new_prev = pd.DataFrame(data_kl, columns=["week_orig", "week_new", "PSI"])
df_kl_new_prev["test_passed"] = df_kl_new_prev["PSI"].apply(
    lambda x: True if x < 0.1 else False
)
df_kl_new_prev

,week_orig,week_new,PSI,test_passed
0,1,2,0.107210,False
1,2,3,0.040858,True
2,3,4,0.051087,True
3,4,5,0.054977,True
4,5,6,0.020872,True
5,6,7,0.090079,True
6,7,8,0.171627,False
7,8,9,0.086136,True
8,9,10,0.019184,True
9,10,11,0.039348,True


In [14]:
kl = []
ref_week = 1
data_kl = []
for j in range(1, 20):
    p = (
        df_duration_per_day[(df_duration_per_day["week"] == j)]["duration"].values
        / df_duration_per_day[(df_duration_per_day["week"] == j)][
            "duration"
        ].values.sum()
    )
    q = (
        df_duration_per_day[(df_duration_per_day["week"] == ref_week)][
            "duration"
        ].values
        / df_duration_per_day[(df_duration_per_day["week"] == ref_week)][
            "duration"
        ].values.sum()
    )
    res = sp.special.kl_div(p, q).sum()
    res += sp.special.kl_div(q, p).sum()
    data_kl.append([ref_week, j + 1, res])

df_kl_orig_prev = pd.DataFrame(data_kl, columns=["week_orig", "week_new", "PSI"])
df_kl_orig_prev["test_passed"] = df_kl_orig_prev["PSI"].apply(
    lambda x: True if x < 0.1 else False
)
df_kl_orig_prev

,week_orig,week_new,PSI,test_passed
0,1,2,0.000000,True
1,1,3,0.107210,False
2,1,4,0.100708,False
3,1,5,0.056635,True
4,1,6,0.057106,True
5,1,7,0.057112,True
6,1,8,0.096630,True
7,1,9,0.035553,True
8,1,10,0.087863,True
9,1,11,0.086282,True


Как и в случае Колмогорова-Смирнова иногда наблюдаются отклонения, но не критичные. 
Значение PSI меньше везде 0.2, т.е. Все находится в пределах разумного.

Выбросы для времени просмотра могут быть обусловлены интересом аудитории к нашумевшим новинкам киноиндустрии.

## Бинарная \ дискретная метрика

В качестве бинарной метрики возьмем число просмотров фильмов жанра "Drama"

In [15]:
df_users_2018["genres"].values[0]

df_2018_drama = pd.DataFrame(df_users_2018[["datetime", "genres"]])
df_2018_drama["drama"] = (
    df_2018_drama["genres"]
    .str.contains("Drama", regex=False)
    .apply(lambda x: 1 if x else 0)
)

df_2018_drama

,datetime,genres,drama
247686,2018-01-01,"Action, Drama, Sci-Fi, Thriller",1
247653,2018-01-01,NOT AVAILABLE,0
248550,2018-01-01,"Animation, Adventure, Comedy, Family",0
247570,2018-01-01,"Biography, Crime, Drama",1
247438,2018-01-01,"Animation, Adventure, Comedy, Family",0
...,...,...,...
520522,2018-12-31,"Action, Adventure, Fantasy, Sci-Fi",0
520553,2018-12-31,"Action, Adventure, Fantasy, Sci-Fi",0
520241,2018-12-31,"Adventure, Comedy, Drama, Family, Fantasy",1
520437,2018-12-31,"Action, Crime, Mystery, Sci-Fi, Thriller",0


In [16]:
df_2018_drama = df_2018_drama.groupby(by="datetime", as_index=False).agg(
    {"drama": "sum"}
)
df_2018_drama

,datetime,drama
0,2018-01-01,385
1,2018-01-02,338
2,2018-01-03,280
3,2018-01-04,272
4,2018-01-05,278
...,...,...
360,2018-12-27,400
361,2018-12-28,1490
362,2018-12-29,1335
363,2018-12-30,1203


In [17]:
df_2018_drama["datetime"] = pd.to_datetime(df_2018_drama["datetime"])
df_2018_drama["week"] = df_2018_drama["datetime"].dt.isocalendar().week
df_2018_drama = df_2018_drama.drop(
    df_2018_drama[df_2018_drama["datetime"] == pd.to_datetime("2018-12-31")].index
)
df_2018_drama

,datetime,drama,week
0,2018-01-01,385,1
1,2018-01-02,338,1
2,2018-01-03,280,1
3,2018-01-04,272,1
4,2018-01-05,278,1
...,...,...,...
359,2018-12-26,454,52
360,2018-12-27,400,52
361,2018-12-28,1490,52
362,2018-12-29,1335,52


### Тест Колмогорова-Смирнова

Сравнение 1 и 2 недели, 2 и 3, и т.д.

In [18]:
data_ks_2samp_new_prev = []
for j in range(1, 20):
    res = sp.stats.ks_2samp(
        df_2018_drama[(df_2018_drama["week"] == j)]["drama"],
        df_2018_drama[(df_2018_drama["week"] == j + 1)]["drama"],
    )
    data_ks_2samp_new_prev.append([j, j + 1, res.pvalue])

data_ks_2samp_new_prev = pd.DataFrame(
    data_ks_2samp_new_prev, columns=["week_orig", "week_new", "pvalue"]
)
data_ks_2samp_new_prev["test_passed"] = data_ks_2samp_new_prev["pvalue"].apply(
    lambda x: True if x > 0.05 else False
)
data_ks_2samp_new_prev

/tmp/ipykernel_78756/1246281358.py:3: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  res = sp.stats.ks_2samp(
/tmp/ipykernel_78756/1246281358.py:3: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  res = sp.stats.ks_2samp(


,week_orig,week_new,pvalue,test_passed
0,1,2,0.212121,True
1,2,3,0.999961,True
2,3,4,0.962704,True
3,4,5,0.962704,True
4,5,6,0.053030,True
5,6,7,0.962704,True
6,7,8,0.212121,True
7,8,9,0.212121,True
8,9,10,0.053030,True
9,10,11,0.053030,True


Сравнение эталонной недели и остальных.

In [19]:
data_ks_2samp_orig_new = []
ref_week = 1

for j in range(1, 20):
    res = sp.stats.ks_2samp(
        df_2018_drama[(df_2018_drama["week"] == ref_week)]["drama"],
        df_2018_drama[(df_2018_drama["week"] == j + 1)]["drama"],
    )
    data_ks_2samp_orig_new.append([ref_week, j + 1, res.pvalue])

data_ks_2samp_orig_new = pd.DataFrame(
    data_ks_2samp_orig_new, columns=["week_orig", "week_new", "pvalue"]
)
data_ks_2samp_orig_new["test_passed"] = data_ks_2samp_orig_new["pvalue"].apply(
    lambda x: True if x > 0.05 else False
)
data_ks_2samp_orig_new

,week_orig,week_new,pvalue,test_passed
0,1,2,0.212121,True
1,1,3,0.575175,True
2,1,4,0.053030,True
3,1,5,0.053030,True
4,1,6,0.962704,True
5,1,7,0.962704,True
6,1,8,0.575175,True
7,1,9,0.212121,True
8,1,10,0.212121,True
9,1,11,0.008159,False


По тесту Колмогорова-Смирнова можно сказать, что число просмотров жанра драма стабильно: распределение совпадает из недели в неделю. 

Что касается выбросов в тесте по эталонной неделе, то тут их можно списать к примеру на то, что на этой неделе вышел какой-нибудь фильм, который захотело посмотреть сразу много человек. 

Можно сделать вывод, что при проверке тестом К-С эталонное сравнение не учитывает плавные нарастнание и спад интереса аудитории к определенному жанру.

### Дивергенция Кульбака-Лейбнера

Проведем те же расчеты, но посчитаем дивергенцию Кульбака-Лейбнера и вычислим индекс стабильности популяции.

Данные отнормируем по общему числу просмотров за неделю для передачи данных на вход функции.

Сравнение 1 и 2 недели, 2 и 3, и т.д.

In [20]:
kl = []
data_kl = []
for j in range(1, 20):
    p = (
        df_2018_drama[(df_2018_drama["week"] == j)]["drama"].values
        / df_2018_drama[(df_2018_drama["week"] == j)]["drama"].values.sum()
    )
    q = (
        df_2018_drama[(df_2018_drama["week"] == j + 1)]["drama"].values
        / df_2018_drama[(df_2018_drama["week"] == j + 1)]["drama"].values.sum(),
    )
    res = sp.special.kl_div(p, q).sum()
    res += sp.special.kl_div(q, p).sum()
    data_kl.append([j, j + 1, res])
df_kl_new_prev = pd.DataFrame(data_kl, columns=["week_orig", "week_new", "PSI"])
df_kl_new_prev["test_passed"] = df_kl_new_prev["PSI"].apply(
    lambda x: True if x < 0.1 else False
)
df_kl_new_prev

,week_orig,week_new,PSI,test_passed
0,1,2,0.047838,True
1,2,3,0.010762,True
2,3,4,0.006405,True
3,4,5,0.012564,True
4,5,6,0.015958,True
5,6,7,0.007277,True
6,7,8,0.030793,True
7,8,9,0.053583,True
8,9,10,0.037777,True
9,10,11,0.018431,True


Сравнение эталонной недели и остальных.

In [21]:
kl = []
ref_week = 1
data_kl = []
for j in range(1, 20):
    p = (
        df_2018_drama[(df_2018_drama["week"] == ref_week)]["drama"].values
        / df_2018_drama[(df_2018_drama["week"] == ref_week)]["drama"].values.sum()
    )
    q = (
        df_2018_drama[(df_2018_drama["week"] == j + 1)]["drama"].values
        / df_2018_drama[(df_2018_drama["week"] == j + 1)]["drama"].values.sum(),
    )
    res = sp.special.kl_div(p, q).sum()
    res += sp.special.kl_div(q, p).sum()
    data_kl.append([ref_week, j + 1, res])
df_kl_ref_new = pd.DataFrame(data_kl, columns=["week_orig", "week_new", "PSI"])
df_kl_ref_new["test_passed"] = df_kl_ref_new["PSI"].apply(
    lambda x: True if x < 0.1 else False
)
df_kl_ref_new

,week_orig,week_new,PSI,test_passed
0,1,2,0.047838,True
1,1,3,0.032625,True
2,1,4,0.030919,True
3,1,5,0.050608,True
4,1,6,0.027870,True
5,1,7,0.010842,True
6,1,8,0.051886,True
7,1,9,0.041839,True
8,1,10,0.043017,True
9,1,11,0.013691,True


Данный тест подтвердил стабильность данной метрики: число просмотров жанра драма стабильно, распределение совпадает из недели в неделю. 
